In [112]:
import sys
from datetime import datetime, timedelta, time
import pandas as pd
import yfinance as yf
import pytz
import holidays

In [113]:
# ---CONFIGURATION CONSTANTS---

MARKET_OPEN = time(9, 15)
MARKET_CLOSE = time(15, 30)
TIMEZONE = pytz.timezone("Asia/Kolkata")

In [114]:
# Holiday calendar for India
INDIA_HOLIDAYS = holidays.India()

In [115]:
# -----HELPER FUNCTIONS-----

def is_trading_day(dt):
    """Return True if given datetime is a valid trading day (Mon-Fri, not a holiday)."""
    if dt.weekday() >= 5:
        return False
    if dt.date() in INDIA_HOLIDAYS:
        return False
    return True

# ----Valid ticker check function-----

def is_valid_ticker(ticker):
    """Checks if a ticker symbol is valid by attempting to fetch its historical data."""
    try:
        # Attempt to download a very small amount of data (e.g., 1 day)
        # for a recent period to check if the ticker exists and has data.
        today = datetime.now()
        start_date = today - timedelta(days=7) # Look back 7 days
        df = yf.download(ticker, start=start_date, end=today, interval="1d", progress=False)
        return not df.empty # If df is not empty, it's likely a valid ticker
    except Exception:
        # Any error during download indicates an invalid ticker
        return False

In [116]:
#--------Valid market hours checking function------

def is_within_market_hours(dt):
    """Check if datetime falls within BSE market hours."""
    if not is_trading_day(dt):
        return False
    return MARKET_OPEN <= dt.time() <= MARKET_CLOSE

In [117]:
#--------Valid market date checking function------

def get_next_valid_market_time(dt):
    dt = dt.astimezone(TIMEZONE)
    while True:
        if not is_trading_day(dt):
            dt = (dt + timedelta(days=1)).replace(
                hour=MARKET_OPEN.hour,
                minute=MARKET_OPEN.minute,
                second=0,
                microsecond=0
            )
            continue
        if dt.time() < MARKET_OPEN:
            dt = dt.replace(hour=MARKET_OPEN.hour,
                            minute=MARKET_OPEN.minute,
                            second=0, microsecond=0)
            return dt
        elif dt.time() > MARKET_CLOSE:
            dt = (dt + timedelta(days=1)).replace(
                hour=MARKET_OPEN.hour,
                minute=MARKET_OPEN.minute,
                second=0, microsecond=0
            )
            continue
        else:
            return dt

In [118]:
def fetch_price_at(ticker, dt):
    """Fetch nearest available stock price for a given datetime, with dynamic interval selection."""
    dt = get_next_valid_market_time(dt)
    now_ist = datetime.now(TIMEZONE)

    # If the requested time is in the future — skip it
    if dt > now_ist:
        return None, "Future time — market data not yet available"

    # Choose interval based on date difference
    days_diff = (now_ist.date() - dt.date()).days
    if days_diff <= 30:
        interval = "1m"   # minute data for recent days
        start = dt - timedelta(minutes=15)
        end = dt + timedelta(minutes=5)
    else:
        interval = "1d"   # daily data for older dates
        start = dt - timedelta(days=5)
        end = dt + timedelta(days=5)

    try:
        df = yf.download(ticker, start=start, end=end, interval=interval, progress=False)
        if df.empty:
            return None, "No data"
        df.index = df.index.tz_convert(TIMEZONE)
        nearest_time = df.index[df.index.get_indexer([dt], method="nearest")[0]]
        price = df.loc[nearest_time, "Close"]
        if isinstance(price, pd.Series):
            price = price.item()
        return nearest_time, round(price, 2)
    except Exception as e:
        return None, f"Error: {e}"


In [119]:
# -----------------------------
# MAIN PROGRAM
# -----------------------------
def main():
    print("\n Welcome to BSE Stock Price Tracker (via Yahoo Finance)\n")
    print("Example:  Ticker: TVSMOTOR.BO | Date: 06 Nov 2025 | Time: 13:33")
    print("------------------------------------------------------------\n")


In [120]:
# -----------------------------
# Welcome message
# -----------------------------
if __name__ == "__main__":
    main()


 Welcome to BSE Stock Price Tracker (via Yahoo Finance)

Example:  Ticker: TVSMOTOR.BO | Date: 06 Nov 2025 | Time: 13:33
------------------------------------------------------------



In [121]:
    # ---- USER INPUT ----
    while True: # Loop until a valid ticker is entered
        ticker = input("Enter stock ticker (e.g., TVSMOTOR.BO): ").strip().upper()
        print(f"Entered Ticker: {ticker}")
        if not ticker:
            print("❌ Error: Ticker cannot be empty. Please re-enter.")
            continue # Ask again
        if not is_valid_ticker(ticker):
            print(f"❌ Warning: '{ticker}' is not a valid stock ticker. Please re-enter.")
            continue # Ask again
        break # Valid ticker, exit loop

    while True: # Loop until a valid date/time is entered
        date_input = input("Enter date (DD Mon YYYY) [Default: today]: ").strip()
        print(f"Entered Date: {date_input if date_input else 'today'}")
        time_input = input("Enter time (HH:MM, 24-hour) [Default: now]: ").strip()
        print(f"Entered Time: {time_input if time_input else 'now'}")

        now_ist = datetime.now(TIMEZONE)
        if not date_input:
            date_input = now_ist.strftime("%d %b %Y")
        if not time_input:
            time_input = now_ist.strftime("%H:%M")

        try:
            base_time = datetime.strptime(f"{date_input}, {time_input}", "%d %b %Y, %H:%M")
            base_time = TIMEZONE.localize(base_time)
        except ValueError:
            print("❌ Invalid date/time format. Please use: '06 Nov 2025, 13:33'. Please re-enter.")
            continue # Re-enter date/time

        current_date_ist = datetime.now(TIMEZONE).date()
        if base_time.date() > current_date_ist:
            days_diff = (base_time.date() - current_date_ist).days
            if days_diff > 30:
                print(f"❌ Error: Requested date '{base_time.strftime('%d %b %Y')}' is more than 30 days in the future. Minute data is not available for dates this far ahead. Please enter a date within the next 30 days from today. Please re-enter date.")
                continue # Re-enter date/time
            else:
                # Future date within 30 days
                print(f"⚠️ Warning: Requested date '{base_time.strftime('%d %b %Y')}' has not yet arrived. Data for future dates might not be available or accurate until the market opens on that day.")
                proceed = input("Do you wish to proceed with this future date? (yes/no): ").strip().lower()
                if proceed != 'yes':
                    continue # Re-enter date/time
        elif base_time.date() < current_date_ist:
            days_since = (current_date_ist - base_time.date()).days
            if days_since > 30:
                print(f"❌ Error: Detailed minute data for '{base_time.strftime('%d %b %Y')}' is only available for the last 30 days. Please enter a date within the last 30 days from today. Please re-enter date.")
                continue # Re-enter date/time

        # New: Check for trading day after initial date validation
        if not is_trading_day(base_time):
            print(f"❌ Error: '{base_time.strftime('%d %b %Y')}' is a weekend or holiday. Market closed. Please select a valid trading weekday (Mon–Fri) that is not a holiday. Please re-enter date.")
            continue # Re-enter date/time

        break # Valid date/time, exit loop

Enter stock ticker (e.g., TVSMOTOR.BO): tvsmotor.bo


/tmp/ipython-input-1847626152.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=today, interval="1d", progress=False)


Entered Ticker: TVSMOTOR.BO
Enter date (DD Mon YYYY) [Default: today]: 21 oct 2025
Entered Date: 21 oct 2025
Enter time (HH:MM, 24-hour) [Default: now]: 12:30
Entered Time: 12:30


In [127]:
    # ---- VALIDATION ----
    # is_trading_day validation is now handled in the input loop for reprompting.

    if not is_within_market_hours(base_time):
        print(f"⚠️ Market is open from {MARKET_OPEN.strftime('%H:%M')} to {MARKET_CLOSE.strftime('%H:%M')} IST.")
        base_time = get_next_valid_market_time(base_time)
        print(f"⏩ Adjusted to next valid market time: {base_time.strftime('%d %b %Y, %H:%M %Z')}")

In [128]:
    # ---- TIME OFFSETS ----
    intervals = [
        ("Base Time", timedelta(minutes=0)),
        ("+1 Minute", timedelta(minutes=1)),
        ("+5 Minutes", timedelta(minutes=5)),
        ("+15 Minutes", timedelta(minutes=15)),
        ("+1 Hour", timedelta(hours=1)),
        ("+2 Hours", timedelta(hours=2)),
        ("+1 Day", timedelta(days=1)),
        ("+2 Days", timedelta(days=2))
    ]

    print("\n" + "-"*60)
    print(f" The stock prices of {ticker} for requested date and time are:  \n----------------")
    print(f"{'Time Offset':<15}{'Market Time':<30}{'Price (INR)':>10}")
    print("-"*60)

    for label, delta in intervals:
        target_dt = base_time + delta
        actual_time, price = fetch_price_at(ticker, target_dt)
        if actual_time:
            print(f"{label:<15}{actual_time.strftime('%d %b %Y %H:%M'): <30}{price:>10}")
        else:
            print(f"{label:<15}{'N/A':<30}{price:>10}")

    print("-"*60)

/tmp/ipython-input-1909417694.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TVSMOTOR.BO']: YFPricesMissingError('possibly delisted; no price data found  (1m 2025-10-21 12:15:00+05:30 -> 2025-10-21 12:35:00+05:30)')
/tmp/ipython-input-1909417694.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TVSMOTOR.BO']: YFPricesMissingError('possibly delisted; no price data found  (1m 2025-10-21 12:16:00+05:30 -> 2025-10-21 12:36:00+05:30)')
/tmp/ipython-input-1909417694.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False)
ERROR:yfinance:
1 Fai


------------------------------------------------------------
 The stock prices of TVSMOTOR.BO for requested date and time are:  
----------------
Time Offset    Market Time                   Price (INR)
------------------------------------------------------------
Base Time      N/A                           No minute data found for this period. This can happen if the market was closed, or if no trading occurred at the exact minute.
+1 Minute      N/A                           No minute data found for this period. This can happen if the market was closed, or if no trading occurred at the exact minute.
+5 Minutes     N/A                           No minute data found for this period. This can happen if the market was closed, or if no trading occurred at the exact minute.
+15 Minutes    N/A                           No minute data found for this period. This can happen if the market was closed, or if no trading occurred at the exact minute.
+1 Hour        N/A                           No

/tmp/ipython-input-1909417694.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False)
